In [ ]:
import os

os.chdir('..')

In [ ]:
from zipfile import ZipFile

import polars as pl

from bhds.aws_basics import AWS_TIMEOUT_SEC, get_aws_dir, get_funding_rate_path_tokens, get_kline_path_tokens
from bhds import aws_funding
from config import Config
from constant import TradeType
from fetcher.binance import BinanceFetcher
from util.network import create_aiohttp_session

In [ ]:
http_proxy = 'http://127.0.0.1:7890'

async with create_aiohttp_session(AWS_TIMEOUT_SEC) as session:
    fetcher = BinanceFetcher(TradeType.um_futures, session, http_proxy)
    df = await fetcher.get_hist_funding_rate(symbol='BTCUSDT', limit=1000)

df

In [ ]:
trade_type = TradeType.um_futures
funding_dir = Config.BINANCE_DATA_DIR / 'aws_data' / get_aws_dir(get_funding_rate_path_tokens(trade_type))
funding_dir = Config.BINANCE_DATA_DIR / 'api_data' / 'funding_rate'/ trade_type.value
funding_files = sorted(funding_dir.glob('*.pqt'))

last_begin_time = []
for funding_file in funding_files:
    df = pl.read_parquet(funding_file)
    symbol = funding_file.stem
    last_begin_time.append({'symbol': symbol, 'last_begin_time': df['candle_begin_time'].max()})

pl.DataFrame(last_begin_time).sort('last_begin_time')

In [ ]:
p = '/mnt/ssd_data/crypto_data/binance_data/aws_data/data/futures/um/monthly/fundingRate/BTCUSDT/BTCUSDT-fundingRate-2024-11.zip'
with ZipFile(p) as f:
    filename = f.namelist()[0]
    lines = f.open(filename).readlines()
if lines[0].decode().startswith('calc_time'):
    print(lines[0])
    lines = lines[1:]

columns = ['candle_begin_time', 'funding_interval_hours', 'funding_rate']
schema = {
    'candle_begin_time': pl.Int64,
    'funding_rate': pl.Float64,
}
df_lazy = pl.scan_csv(lines, has_header=False, new_columns=columns, schema_overrides=schema)
df_lazy = df_lazy.drop('funding_interval_hours')
candle_begin_time = pl.col('candle_begin_time') - pl.col('candle_begin_time') % (60 * 60 * 1000)
df_lazy = df_lazy.with_columns(candle_begin_time.cast(pl.Datetime('ms')).dt.replace_time_zone('UTC'))

df_lazy.collect()